In [ ]:
from __future__ import annotations
import pandas as pd
import requests
import json
from google.auth import default
from google.auth.transport.requests import Request

In [ ]:
input_image_link = "gs://mdm-data-preprod-poc-product-similarity-euw1/images/TestChatBot/test_chaise_jaune.jpg"

In [ ]:
def get_json(link):
    return{
    "requests": [
      {
        "image": {
          "source": {
            "gcsImageUri": link
          }
        },
        "features": [
          {
            "type": "PRODUCT_SEARCH",
            "maxResults": 10
          }
        ],
        "imageContext": {
          "productSearchParams": {
            "productSet": "projects/mdm-data-prod/locations/europe-west1/productSets/product_set2",
            "productCategories": [
              "homegoods-v2"
            ],
            "filter": ""
          }
        }
      }
    ]
  }

In [ ]:
get_json(input_image_link)

In [ ]:
def get_mkp_products(link):

    url = "https://vision.googleapis.com/v1/images:annotate"

    # start_time = time.time()

    # print("Récupération des credentials...")
    credentials, project = default()
    # print(f"Credentials obtenus en {time.time() - start_time:.2f}s")

    # print("Refresh du token...")
    credentials.refresh(Request())
    access_token = credentials.token
    # print(f"Token refreshé en {time.time() - start_time:.2f}s")

    # Headers
    headers = {
        "Authorization": f"Bearer {access_token}",
        "x-goog-user-project": "mdm-data-prod",
        "Content-Type": "application/json; charset=utf-8"
    }

    data = get_json(link)
    response = requests.post(url, headers=headers, json=data, timeout=(5, 30))

    # print(f"Réponse reçue en {time.time() - start_time:.2f}s")
    # print(f"Status Code: {response.status_code}")
    # print(f"Response: {response.text}")

    return response.text

In [ ]:
response_text = get_mkp_products(input_image_link)

In [ ]:
def quick_parse(response_text):
    data = json.loads(response_text)
    
    results_data = []
    ranking = 1
    
    for response in data.get('responses', []):
        for result in response.get('productSearchResults', {}).get('results', []):
            if 'product' in result and 'displayName' in result['product']:
                results_data.append({
                    'product': result['product']['displayName'],
                    'ranking': ranking,
                    'similarity_score': result.get('score', 0.0)
                })
                ranking += 1
    
    return pd.DataFrame(results_data)

In [ ]:
df_output = quick_parse(response_text)

In [ ]:
list(df_output["product"])